# Notebook for text extraction on image
Inga Ulusoy, SSC, July 2022

In [ ]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    !pip install setuptools==61 -qqq
    # install misinformation
    !pip install git+https://github.com/ssciwr/misinformation.git@set-up-colab -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [ ]:
import os
from IPython.display import Image, display
import misinformation

In [ ]:
images = misinformation.find_files(
    path="drive/MyDrive/misinformation-data/", limit=1000
)

In [ ]:
for i in images[0:10]:
    display(Image(filename=i))

In [ ]:
mydict = misinformation.utils.initialize_dict(images[0:10])

In [ ]:
mydict

# google cloud vision API
First 1000 images per month are free.

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "../data/misinformation-campaign-981aa55a3b13.json"

## Inspect the elements per image

In [ ]:
misinformation.explore_analysis(mydict, identify="text-on-image")

## Or directly analyze for further processing

In [ ]:
for key in mydict:
    print(key)
    mydict[key] = misinformation.text.TextDetector(mydict[key]).analyse_image()

In [ ]:
print(mydict["109237S_spa"]["text_clean"])

## Convert to dataframe and write csv

In [ ]:
outdict = misinformation.utils.append_data_to_dict(mydict)
df = misinformation.utils.dump_df(outdict)

In [ ]:
# check the dataframe
df.head(10)

In [ ]:
# Write the csv
df.to_csv("drive/MyDrive/misinformation-data/data_out.csv")